In [77]:
from __future__ import print_function
import numpy as np


class RBM:
    def __init__(self, num_visible, num_hidden):
        self.num_hidden = num_hidden
        self.num_visible = num_visible
        self.debug_print = True

        # Initialize a weight matrix, of dimensions (num_visible x num_hidden), using
        # a uniform distribution between -sqrt(6. / (num_hidden + num_visible))
        # and sqrt(6. / (num_hidden + num_visible)). One could vary the 
        # standard deviation by multiplying the interval with appropriate value.
        # Here we initialize the weights with mean 0 and standard deviation 0.1. 
        # Reference: Understanding the difficulty of training deep feedforward 
        # neural networks by Xavier Glorot and Yoshua Bengio
        np_rng = np.random.RandomState(1234)

        self.weights = np.asarray(
            np_rng.uniform(
                low=-0.1 * np.sqrt(6. / (num_hidden + num_visible)),
                high=0.1 * np.sqrt(6. / (num_hidden + num_visible)),
                size=(num_visible, num_hidden)))

        # Insert weights for the bias units into the first row and first column.
        self.weights = np.insert(self.weights, 0, 0, axis=0)
        self.weights = np.insert(self.weights, 0, 0, axis=1)

    def train(self, data, max_epochs=1000, learning_rate=0.1):
        """
        argmin_W (-log p(W | v))

        Train the machine.

        Parameters
        ----------
        data: A matrix where each row is a training example consisting of the states of visible units.    
        """

        num_examples = data.shape[0]

        # Insert bias units of 1 into the first column.
        data = np.insert(data, 0, 1, axis=1)

        for epoch in range(max_epochs):
            # Clamp to the data and sample from the hidden units. 
            # (This is the "positive CD phase", aka the reality phase.)
            # 1) p( h | V ; W )

            pos_hidden_activations = np.dot(data, self.weights)
            pos_hidden_probs = self._logistic(pos_hidden_activations)
            pos_hidden_probs[:, 0] = 1  # Fix the bias unit.

            # 2) H ~ p( h | V ; W )
            pos_hidden_states = pos_hidden_probs > np.random.rand(
                num_examples, self.num_hidden + 1)
            # Note that we're using the activation *probabilities* of the hidden states, not the hidden states       
            # themselves, when computing associations. We could also use the states; see section 3 of Hinton's 
            # "A Practical Guide to Training Restricted Boltzmann Machines" for more.

            # W+ = V^T * H  ( positive associations: outer product)
            pos_associations = np.dot(data.T, pos_hidden_probs)

            # Reconstruct the visible units and sample again from the hidden units.
            # (This is the "negative CD phase", aka the daydreaming phase.)

            # v' = p( v | H; W)
            neg_visible_activations = np.dot(pos_hidden_states, self.weights.T)
            neg_visible_probs = self._logistic(neg_visible_activations)
            neg_visible_probs[:, 0] = 1  # Fix the bias unit.

            # h' = p( h | v; W)
            neg_hidden_activations = np.dot(neg_visible_probs, self.weights)
            neg_hidden_probs = self._logistic(neg_hidden_activations)
            # Note, again, that we're using the activation *probabilities* when computing associations, not the states 
            # themselves.

            # W- = V'^T * H'
            neg_associations = np.dot(neg_visible_probs.T, neg_hidden_probs)

            # Update weights.
            # W_t - W_{t-1} += alpha * ( W+ - W- ) / N
            self.weights += learning_rate * (
                (pos_associations - neg_associations) / num_examples)

            error = np.sum((data - neg_visible_probs)**2)
            if self.debug_print:
                print("Epoch %s: error is %s" % (epoch, error))

    def run_visible(self, data):
        """
        p( h | v ; W )

        Assuming the RBM has been trained (so that weights for the network have been learned),
        run the network on a set of visible units, to get a sample of the hidden units.

        Parameters
        ----------
        data: A matrix where each row consists of the states of the visible units.

        Returns
        -------
        hidden_states: A matrix where each row consists of the hidden units activated from the visible
        units in the data matrix passed in.
        """

        num_examples = data.shape[0]

        # Create a matrix, where each row is to be the hidden units (plus a bias unit)
        # sampled from a training example.
        hidden_states = np.ones((num_examples, self.num_hidden + 1))

        # Insert bias units of 1 into the first column of data.
        data = np.insert(data, 0, 1, axis=1)

        # Calculate the activations of the hidden units.
        hidden_activations = np.dot(data, self.weights)
        # Calculate the probabilities of turning the hidden units on.
        hidden_probs = self._logistic(hidden_activations)
        # Turn the hidden units on with their specified probabilities.
        hidden_states[:, :] = hidden_probs > np.random.rand(
            num_examples, self.num_hidden + 1)
        # Always fix the bias unit to 1.
        # hidden_states[:,0] = 1

        # Ignore the bias units.
        return hidden_probs[:, 1:], hidden_states[:, 1:]
    
    # TODO: Remove the code duplication between this method and `run_visible`?
    def run_hidden(self, data):
        """
        p(v | h; W) = ?

        Assuming the RBM has been trained (so that weights for the network have been learned),
        run the network on a set of hidden units, to get a sample of the visible units.

        Parameters
        ----------
        data: A matrix where each row consists of the states of the hidden units.

        Returns
        -------
        visible_states: A matrix where each row consists of the visible units activated from the hidden
        units in the data matrix passed in.
        """

        num_examples = data.shape[0]

        # Create a matrix, where each row is to be the visible units (plus a bias unit)
        # sampled from a training example.
        visible_states = np.ones((num_examples, self.num_visible + 1))

        # Insert bias units of 1 into the first column of data.
        data = np.insert(data, 0, 1, axis=1)

        # Calculate the activations of the visible units.
        visible_activations = np.dot(data, self.weights.T)
        # Calculate the probabilities of turning the visible units on.
        visible_probs = self._logistic(visible_activations)
        # Turn the visible units on with their specified probabilities.
        visible_states[:, :] = visible_probs > np.random.rand(
            num_examples, self.num_visible + 1)
        # Always fix the bias unit to 1.
        # visible_states[:,0] = 1

        # Ignore the bias units.
        return visible_probs[:,1:], visible_states[:, 1:]

    def daydream(self, num_samples):
        """
        v0 ~ N()
        h1 ~ p( h | v0 ; W)
        v1 ~ p( v | h1 ; W)
        h2 ~ p( h | v1 ; W)
        v2 ~ p( v | h2 ; W)
        etc.

        Randomly initialize the visible units once, and start running alternating Gibbs sampling steps
        (where each step consists of updating all the hidden units, and then updating all of the visible units),
        taking a sample of the visible units at each step.
        Note that we only initialize the network *once*, so these samples are correlated.

        Returns
        -------
        samples: A matrix, where each row is a sample of the visible units produced while the network was
        daydreaming.
        """

        # Create a matrix, where each row is to be a sample of of the visible units 
        # (with an extra bias unit), initialized to all ones.
        samples = np.ones((num_samples, self.num_visible + 1))

        # Take the first sample from a uniform distribution.
        samples[0, 1:] = np.random.rand(self.num_visible)

        # Start the alternating Gibbs sampling.
        # Note that we keep the hidden units binary states, but leave the
        # visible units as real probabilities. See section 3 of Hinton's
        # "A Practical Guide to Training Restricted Boltzmann Machines"
        # for more on why.
        for i in range(1, num_samples):
            visible = samples[i - 1, :]

            # Calculate the activations of the hidden units.
            hidden_activations = np.dot(visible, self.weights)
            # Calculate the probabilities of turning the hidden units on.
            hidden_probs = self._logistic(hidden_activations)
            # Turn the hidden units on with their specified probabilities.
            hidden_states = hidden_probs > np.random.rand(self.num_hidden + 1)
            # Always fix the bias unit to 1.
            hidden_states[0] = 1

            # Recalculate the probabilities that the visible units are on.
            visible_activations = np.dot(hidden_states, self.weights.T)
            visible_probs = self._logistic(visible_activations)
            visible_states = visible_probs > np.random.rand(self.num_visible +
                                                            1)
            samples[i, :] = visible_states

        # Ignore the bias units (the first column), since they're always set to 1.
        return samples[:, 1:]

    def _logistic(self, x):
        return 1.0 / (1 + np.exp(-x))

In [78]:
rbm = RBM(num_visible=6, num_hidden=2)
training_data = np.array(
    [[1, 1, 1, 0, 0, 0], 
     [1, 0, 1, 0, 0, 0], 
     [1, 1, 1, 0, 0, 0],
     [0, 0, 1, 1, 1, 0], 
     [0, 0, 1, 1, 0, 0], 
     [0, 0, 1, 1, 1, 0]])

rbm.train(training_data, max_epochs=5000)
print(rbm.weights)


Epoch 0: error is 9.02391497154
Epoch 1: error is 8.79692204931
Epoch 2: error is 8.66667467083
Epoch 3: error is 8.38165627485
Epoch 4: error is 8.18102076896
Epoch 5: error is 7.89510198826
Epoch 6: error is 7.81462822311
Epoch 7: error is 7.80622118732
Epoch 8: error is 7.60975461534
Epoch 9: error is 7.37801135359
Epoch 10: error is 7.43910613822
Epoch 11: error is 7.05581660774
Epoch 12: error is 7.30788724825
Epoch 13: error is 7.14661870785
Epoch 14: error is 6.91777473124
Epoch 15: error is 7.06360813951
Epoch 16: error is 7.02792379612
Epoch 17: error is 6.49135756764
Epoch 18: error is 6.72542941396
Epoch 19: error is 6.57589155772
Epoch 20: error is 6.69032874921
Epoch 21: error is 6.84870780992
Epoch 22: error is 6.6235474011
Epoch 23: error is 6.18564297075
Epoch 24: error is 6.2544828059
Epoch 25: error is 6.68578084223
Epoch 26: error is 6.40005223563
Epoch 27: error is 6.27384057257
Epoch 28: error is 6.22969203555
Epoch 29: error is 6.38997975615
Epoch 30: error is 6.1

Epoch 895: error is 1.11499908769
Epoch 896: error is 1.35540868597
Epoch 897: error is 1.3550417745
Epoch 898: error is 1.35468943729
Epoch 899: error is 1.48905287574
Epoch 900: error is 1.35512548645
Epoch 901: error is 1.35475553065
Epoch 902: error is 1.35440041252
Epoch 903: error is 1.35405958171
Epoch 904: error is 1.35373250491
Epoch 905: error is 1.35341866556
Epoch 906: error is 1.35311756351
Epoch 907: error is 1.35282871475
Epoch 908: error is 1.35255165106
Epoch 909: error is 1.35228591966
Epoch 910: error is 1.12224176161
Epoch 911: error is 1.35280339247
Epoch 912: error is 1.11595767737
Epoch 913: error is 1.35333917159
Epoch 914: error is 1.35303485359
Epoch 915: error is 1.11093752672
Epoch 916: error is 1.35358426713
Epoch 917: error is 1.35326686432
Epoch 918: error is 1.74427234127
Epoch 919: error is 1.35254169279
Epoch 920: error is 1.11586018347
Epoch 921: error is 1.35307035337
Epoch 922: error is 1.35275983224
Epoch 923: error is 1.11078133133
Epoch 924: erro

Epoch 1831: error is 2.23075530298
Epoch 1832: error is 1.35903346988
Epoch 1833: error is 1.35821221799
Epoch 1834: error is 0.873228015486
Epoch 1835: error is 1.35886472452
Epoch 1836: error is 1.35805037998
Epoch 1837: error is 1.35726405544
Epoch 1838: error is 2.3037782951
Epoch 1839: error is 1.3579389373
Epoch 1840: error is 1.35715767216
Epoch 1841: error is 0.873643432547
Epoch 1842: error is 1.35783662075
Epoch 1843: error is 1.35706011928
Epoch 1844: error is 0.873091653281
Epoch 1845: error is 1.35774283804
Epoch 1846: error is 0.871110576474
Epoch 1847: error is 0.867741490608
Epoch 1848: error is 1.35987375023
Epoch 1849: error is 1.35903084974
Epoch 1850: error is 0.867412932665
Epoch 1851: error is 1.35967561797
Epoch 1852: error is 1.35884055796
Epoch 1853: error is 2.234933478
Epoch 1854: error is 0.872135559891
Epoch 1855: error is 0.868742282379
Epoch 1856: error is 0.865453224208
Epoch 1857: error is 0.862264476092
Epoch 1858: error is 1.3629442564
Epoch 1859: err

Epoch 3103: error is 0.686355895786
Epoch 3104: error is 0.686300952801
Epoch 3105: error is 0.686246567814
Epoch 3106: error is 0.686192732337
Epoch 3107: error is 0.686139438047
Epoch 3108: error is 0.686086676777
Epoch 3109: error is 0.68603444052
Epoch 3110: error is 0.685982721419
Epoch 3111: error is 0.685931511766
Epoch 3112: error is 0.685880803998
Epoch 3113: error is 0.685830590692
Epoch 3114: error is 0.685780864564
Epoch 3115: error is 0.685731618466
Epoch 3116: error is 0.68568284538
Epoch 3117: error is 0.685634538416
Epoch 3118: error is 0.68558669081
Epoch 3119: error is 1.55348315716
Epoch 3120: error is 0.685868964446
Epoch 3121: error is 0.685817563095
Epoch 3122: error is 0.685766667658
Epoch 3123: error is 0.685716270703
Epoch 3124: error is 0.685666364935
Epoch 3125: error is 0.685616943191
Epoch 3126: error is 0.685567998436
Epoch 3127: error is 1.55292645095
Epoch 3128: error is 0.685856394818
Epoch 3129: error is 0.685803834924
Epoch 3130: error is 0.6857517993

Epoch 4188: error is 0.674374359272
Epoch 4189: error is 0.67436791894
Epoch 4190: error is 0.674361492455
Epoch 4191: error is 0.674355079762
Epoch 4192: error is 0.674348680806
Epoch 4193: error is 0.674342295536
Epoch 4194: error is 1.60596601531
Epoch 4195: error is 0.674381730257
Epoch 4196: error is 0.674374968552
Epoch 4197: error is 1.61388939467
Epoch 4198: error is 0.674310540923
Epoch 4199: error is 0.674304238003
Epoch 4200: error is 0.674297948423
Epoch 4201: error is 0.674291672139
Epoch 4202: error is 0.674285409105
Epoch 4203: error is 2.08354056094
Epoch 4204: error is 0.674410659716
Epoch 4205: error is 0.674402603013
Epoch 4206: error is 0.674394668778
Epoch 4207: error is 0.67438685103
Epoch 4208: error is 0.674379144078
Epoch 4209: error is 0.674371542503
Epoch 4210: error is 0.674364041151
Epoch 4211: error is 0.674356635114
Epoch 4212: error is 0.674349319726
Epoch 4213: error is 0.674342090547
Epoch 4214: error is 0.674334943353
Epoch 4215: error is 1.6070074722

In [79]:
observation = np.array([[0, 0, 0, 1, 1, 0]])

_, hidden = rbm.run_visible(observation)
hidden


array([[ 1.,  0.]])

In [80]:

_, generated_observation = rbm.run_hidden(np.array([[1, 0]]))

generated_observation

array([[ 0.,  0.,  1.,  1.,  1.,  0.]])

# Response of hidden neurons to train data

In [81]:
%precision 4
np.set_printoptions(suppress=True)

hidden_probs, hidden_states=rbm.run_visible(training_data)
hidden_probs[:,1:]


array([[ 1.    ],
       [ 1.    ],
       [ 1.    ],
       [ 0.0116],
       [ 0.943 ],
       [ 0.0116]])

In [84]:
rbm.run_hidden(hidden_probs)

(array([[ 0.9894,  0.6634,  0.9999,  0.0139,  0.001 ,  0.0014],
        [ 0.9892,  0.6606,  0.9999,  0.0141,  0.001 ,  0.0014],
        [ 0.9894,  0.6634,  0.9999,  0.0139,  0.001 ,  0.0014],
        [ 0.0006,  0.002 ,  0.9988,  0.9992,  0.9712,  0.0016],
        [ 0.0233,  0.0077,  1.    ,  0.9747,  0.0379,  0.0001],
        [ 0.0006,  0.002 ,  0.9988,  0.9992,  0.9712,  0.0016]]),
 array([[ 1.,  1.,  1.,  0.,  0.,  0.],
        [ 1.,  0.,  1.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  1.,  1.,  0.],
        [ 0.,  0.,  1.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  1.,  0.,  0.]]))

In [85]:
training_data

array([[1, 1, 1, 0, 0, 0],
       [1, 0, 1, 0, 0, 0],
       [1, 1, 1, 0, 0, 0],
       [0, 0, 1, 1, 1, 0],
       [0, 0, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 0]])